In [184]:
INPUT_FILE = "example-1.txt"

In [185]:
INPUT_FILE = "example-2.txt"

In [186]:
INPUT_FILE = "example-3.txt"

In [187]:
INPUT_FILE = "example-4.txt"

In [188]:
INPUT_FILE = "example-5.txt"

In [189]:
INPUT_FILE = "example-6.txt"

In [190]:
INPUT_FILE = "example-7.txt"

In [191]:
INPUT_FILE = "example-8.txt"

In [192]:
INPUT_FILE = "input.txt"

In [193]:
def hexToBinary(hex):
    result = ""
    for char in hex:
        value = int(char, 16)
        result += "{:04b}".format(value)
    return result

with open(INPUT_FILE, "r") as file:
    input = hexToBinary(file.read().strip("\n"))

In [194]:
parsePacket = None

def parseLiteralPacket(start):
    index = start
    number = ""
    while True:
        number += input[index + 1: index + 5]
        if input[index] == '0':
            break
        index += 5

    return int(number, 2), index + 5

def parseOperatorPacket(start):
    def parseChildPacket(start):
        childPacket, end = parsePacket(start)
        childPackets.append(childPacket)
        return end     

    lengthTypeId = input[start]
    childPackets = []
    if lengthTypeId == '0':
        index = start + 16
        end = index + int(input[start + 1: start + 16], 2)
        while index < end:
            index = parseChildPacket(index)
        return childPackets, end
    else:
        totalChildPackets = int(input[start + 1: start + 12], 2)
        index = start + 12
        for _ in range(totalChildPackets):
            index = parseChildPacket(index)
        return childPackets, index

def parsePacket(start=0):
    version = int(input[start: start + 3], 2)
    typeId = int(input[start + 3: start + 6], 2)
    
    match typeId:
        case 4: content, end = parseLiteralPacket(start + 6)
        case _: content, end = parseOperatorPacket(start + 6)

    return {
        "version": version,
        "typeId": typeId,
        "content": content
    }, end

In [195]:
def versionSum(packet):
    current = packet['version']
    if packet['typeId'] != 4:
        current += sum(map(versionSum, packet['content']))
    return current


packet, end = parsePacket()

versionSum(packet)

871

In [196]:
import functools

evaluatePacket = None

def evaluateSum(packet):
    return sum(map(evaluatePacket, packet['content']))

def evaluateProduct(packet):
    return functools.reduce(lambda x, y: y * x, map(evaluatePacket, packet['content']))

def evaluateMinimum(packet):
    return min(map(evaluatePacket, packet['content']))

def evaluateMaximum(packet):
    return max(map(evaluatePacket, packet['content']))

def evaluateLiteral(packet):
    return packet['content']

def evaluateGraterThan(packet):
    return 1 if evaluatePacket(packet['content'][0]) > \
        evaluatePacket(packet['content'][1]) else 0

def evaluateLessThan(packet):
    return 1 if evaluatePacket(packet['content'][0]) < \
        evaluatePacket(packet['content'][1]) else 0

def evaluateEqualTo(packet):
    return 1 if evaluatePacket(packet['content'][0]) == \
        evaluatePacket(packet['content'][1]) else 0

def evaluatePacket(packet):
    match packet['typeId']:
        case 0: return evaluateSum(packet)
        case 1: return evaluateProduct(packet)
        case 2: return evaluateMinimum(packet)
        case 3: return evaluateMaximum(packet)
        case 4: return evaluateLiteral(packet)
        case 5: return evaluateGraterThan(packet)
        case 6: return evaluateLessThan(packet)
        case 7: return evaluateEqualTo(packet)


In [197]:
packet, end = parsePacket()
evaluatePacket(packet)

68703010504